## 1.互相关运算

In [ ]:
#互相关运算
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X,K): #@save
    """计算二维互相关运算，输入张量为X，卷积核张量为K"""
    h,w=K.shape
    Y=torch.zeros((X.shape[0]-h+1,X.shape[1]-w+1)) #计算输出二维张量的维度
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(X[i:i+h,j:j+w]*K).sum() #核窗口中局部张量与核相乘，即元素对应相乘后相加；h、w是核的高度、宽度；数组切片包含开始位置不包含截止位置
    return Y

#卷积层
class Conv2D(nn.Module): #二维卷积层函数。卷积层中两个被训练的参数：1.卷积核权重weight 2.标量偏置bias
    def __init__(self,kernel_size): #构造函数
        super.__init__()
        self.weight=nn.Parameter(torch.rand(kernel_size)) #模型参数weight。nn.Parameter会自动被认为是module的可训练参数，即加入到parameter()这个迭代器中去nn。Parameter对象的requires_grad属性的默认值是True
        self.bias=nn.Parameter(torch.zeros(1))#偏置参数bias
    
    def forward(self,x):#前向传播函数
        return corr2d(x,self.weight)+self.bias #调用corr2d()计算输入与权重的卷积，再加上偏置作为输出
    
#图像中目标的边缘检测
X=torch.ones(6,8) #构造一个 6×8 像素的黑白图像。中间四列为黑色（ 0 ），其余像素为白色（ 1 ）
X[:,2:6]=0
K=torch.tensor([[1.0,-1.0]]) #1x2卷积核K，进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零
Y=corr2d(X,K) #输出Y中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘
corr2d(X.t(),K) #将输入X转置后，原有卷积核K无法检测到水平边缘(只能检测到垂直边缘)

#学习卷积核
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False) #使用内置卷积核函数Conv2d，输入通道、输出通道均设置为1，卷积核大小设置为1x2，忽略偏置

X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
lr=3e-2

for i in range(10):
    Y_hat=conv2d(X) #使用内置卷积核与输入X进行运算后，得到输出Y_hat
    l=(Y_hat-Y) #计算Y_hat与真实值之间的平方误差损失函数
    conv2d.zero_grad() #梯度清零
    l.sum(.backward()) #损失函数求梯度
    conv2d.weight.data[:]-=lr*conv2d.weight.grad #迭代更新卷积核权重weight
    if (i+1)%2==0: #每两轮迭代打印信息
        print(f'epoch {i+1},loss {l.sum(:.3f)}')
        
conv2d.weight.data.reshape((1,2)) #检查学习到的卷积核的权重张量，与自定义权重张量相差不大


## 2.填充和步幅

In [ ]:
import torch
from torch import nn

def comp_conv2d(conv2d,X): # 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
    X=X.reshape((1,1)+X.shape) #（1，1）表示批量大小和通道数都是1。这里(1, 1) + X.shape是什么目的？？？
    Y=conv2d(X)
    return Y.reshape(Y.shape[2:])  #??? # 省略前两个维度：批量大小和通道

conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1)  #padding=1表示每边都填充了1行或1列，因此总共添加了2行2列
X=torch.rand(size=(8,8)) #构造输入矩阵X
comp_conv2d(conv2d,X).shape #查看经过comp_conv2d()后输出的shape

conv2d=nn.Conv2d(1,1,kernel_size=(5,3),padding=(2,1)) #使用高度5宽度3的卷积核，高宽两边填充分别为2，1,使得输出和输入具有相同的高度和宽度。𝑝ℎ=(𝑘ℎ−1)/2 和 𝑝𝑤=(𝑘𝑤−1)/2
comp_conv2d(conv2d,X).shape

conv2d=nn.Conv2d(1,1,kernel_size=3,padding=1,stride=abs2) #卷积核大小3x3，各边填充1，垂直/水平步幅2
comp_conv2d(conv2d,X).shape #输入的高度和宽度将减半

conv2d=nn.Conv2d(1,1,kernel_size=(3,5),padding=(0,1),stride=(3,4)) #卷积核高/宽为3x5；填充高0，宽两边各1；步幅垂直3，水平4
comp_conv2d(conv2d,X).shape


## 3.多输入多输出通道

In [ ]:
import torch
from d2l import torch as d2l

#多输入通道单输出通道
def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

#多输入通道多输出通道
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

K=torch.stack((K,K+1,K+2),0)
K.shape

corr2d_multi_in_out(X,K)

#1x1卷积层
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w=X.shape
    c_o=K.shape[0]
    X=X.reshape((c_i))
